In [2]:
import warnings
import sys
import scipy.io
import pickle
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import EntropyHub as eh
import argparse
import antropy as an
from zipfile import ZipFile
from typing import TypeVar
from typing import Dict, List
from typing import Dict
from typing import Any, Callable
from tqdm import tqdm
from time import sleep
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, LeaveOneOut, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut, LeavePGroupsOut
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from postprocess_significant_electrodes_users import caculate_mode_users
from postprocess_significant_electrodes_all import caculate_mode_all
from pathlib import Path
from pandas import Series
from pandas import read_pickle, DataFrame
from pandas import read_pickle
from pandas import DataFrame, set_option, read_pickle
from pandas import DataFrame
from pandas.core.frame import DataFrame
from pandas._config.config import set_option
from os import getcwd
from model import model_svc_wide, wide_params, model_svc, model_mlp
from model import model_svc
from model import model_rfc, model_mlp, model_svc, model_knn
from mne.io import read_raw_cnt
from mne import make_fixed_length_epochs
from mne.epochs import Epochs
from math import gamma
from math import floor
from joblib import dump, load
from itertools import product
from itertools import combinations
from itertools import chain, combinations
from itertools import accumulate
from IPython.display import display
from IPython.core.display import display
from datetime import datetime

%cd /home/matej/2-fer/uuzop/eeg-driver-fatigue-detection
FREQ = 1000
num_users = 12
EPOCH_SECONDS = 1

signal_offset = -20
SIGNAL_FILE_DURATION_SECONDS = 600
SIGNAL_DURATION_SECONDS_DEFAULT = 300

FATIGUE_STR = "fatigue"
NORMAL_STR = "normal"

channels_all = ["HEOL", "HEOR", "FP1", "FP2", "VEOU", "VEOL", "F7", "F3", "FZ", "F4", "F8", "FT7", "FC3", "FCZ", "FC4", "FT8", "T3", "C3", "CZ", "C4", "T4", "TP7", "CP3", "CPZ", "CP4", "TP8", "A1", "T5", "P3", "PZ", "P4", "T6", "A2", "O1", "OZ", "O2", "FT9", "FT10", "PO1", "PO2"]

channels_good = ["FP1", "FP2", "F7", "F3", "FZ", "F4", "F8", "FT7", "FC3", "FCZ", "FC4", "FT8", "T3", "C3", "CZ", "C4", "T4", "TP7", "CP3", "CPZ", "CP4", "TP8", "T5", "P3", "PZ", "P4", "T6", "O1", "OZ", "O2"]

channels_bad = list(set(channels_all) - set(channels_good))

channels_ignore = []

PAPER_G = 2 ** (-5)
PAPER_C = 2 ** (-1)
PAPER_RFC_TREES = 500
PAPER_RFC_INPUT_VARIABLES = 22
PAPER_BF_HIDDEN = 22

entropy_names = ["PE", "AE", "SE", "FE"]

states = [NORMAL_STR, FATIGUE_STR]

# [PE_FP1, PE_FP2, ... , PE_C3, AE_FP1, AE_FP2, ..., FE_C3]
entropy_channel_combinations = ["{}_{}".format(entropy, channel) for entropy in entropy_names for channel in channels_good]


/home/matej/2-fer/uuzop/eeg-driver-fatigue-detection


In [3]:
PATH_CWD = Path(getcwd())
PATH_DATA = Path(PATH_CWD, "data")
PATH_REPORT = Path(PATH_DATA, "reports")
PATH_DATASET = Path(PATH_DATA, "dataset")
PATH_DATA_MODEL = Path(PATH_DATA, "models")

PATH_DATA_DATAFRAME = Path(PATH_DATA, "dataframes")
PATH_DATASET_MAT = Path(PATH_DATASET, "mat")
PATH_DATASET_CNT = Path(PATH_DATASET, "cnt")

PATH_ZIP_CNT = Path(PATH_DATASET, "5202739.zip")
PATH_ZIP_MAT = Path(PATH_DATASET, "5202751.zip")

## File saver

In [4]:
METADATA_FIELD_TAGS = "user.xdg.tags"
METADATA_FIELD_COMMENT = "user.xdg.comment"


def load_model(path: Path) -> GridSearchCV:
    return pickle.loads(load(path))


def load_dataframe(path: Path) -> DataFrame:
    return read_pickle(path)


def save_to_file(
    obj: object,
    path: Path,
    file_saver: Callable[[object, str], Any] = lambda obj, filename: dump(obj, filename),
):
    file_saver(obj, str(path))
    print("Saved file:\n", str(path))


def save_to_file_with_metadata(
    obj: object,
    dir: Path,
    basename: str,
    extension: str,
    file_saver: Callable[[object, str], Any] = lambda obj, filename: dump(obj, filename),
    metadata: dict = {},
):
    MAX_FILENAME_LEN = 255
    timestamp = datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
    metadata_str = dict_to_string(metadata)
    metadata_bytes = dict_to_byte_metadata(metadata)
    filename = "-".join([basename, timestamp, metadata_str + extension]).lower()[:MAX_FILENAME_LEN]
    file_path = str(Path(dir, filename))

    save_to_file(obj, file_path, file_saver)
    os.setxattr(file_path, METADATA_FIELD_TAGS, metadata_bytes)
    return file_path


def save_model(model, model_name, score, directory: Path, metadata={}, name_tag=""):
    basename = "{model_name}-{score:.4f}-{name_tag}".format(model_name=model_name, score=score, name_tag=name_tag)
    file_saver = lambda model, filename: dump(pickle.dumps(model), filename)
    return save_to_file_with_metadata(model, directory, basename, ".model", file_saver, metadata)


def save_df_to_disk(df: DataFrame, is_complete_train: bool, dir: Path, name_tag: str, metadata={}):
    data_type_str = "complete" if is_complete_train else "partial"
    basename = "-".join([data_type_str, name_tag])
    metadata = {} if is_complete_train else metadata
    file_saver = lambda df, filename: df.to_pickle(str(filename))
    return save_to_file_with_metadata(df, dir, basename, ".pkl", file_saver, metadata=metadata)
    


def save_npy_to_disk(ndarray: np.ndarray, dir: Path, name_tag: str, metadata: dict = {}):
    file_saver = lambda ndarray, filename: np.save(str(filename), ndarray)
    return save_to_file_with_metadata(ndarray, dir, name_tag, ".npy", file_saver, metadata)


# Functions

In [5]:
T = TypeVar("T")


def get_timestamp():
    return datetime.today().strftime("%Y-%m-%d-%H-%M-%S")


def dict_apply_procedture(old_dict: Dict[str, T], procedure) -> Dict[str, T]:
    return {k: procedure(v) for k, v in old_dict.items()}


def min_max_dataframe(df: DataFrame):
    return DataFrame(min_max_scaler(df))


def standard_scale_dataframe(df: DataFrame):
    return DataFrame(standard_scaler(df))


standard_scaler = preprocessing.StandardScaler().fit_transform


def standard_scaler_1d(x: np.ndarray) -> np.ndarray:
    return standard_scaler(x.reshape(-1, 1)).reshape(1, -1).squeeze()


min_max_scaler = preprocessing.MinMaxScaler((-1, 1)).fit_transform


def min_max_scaler_1d(x):
    return min_max_scaler(x.reshape(-1, 1)).reshape(1, -1).squeeze()


def isnull_any(df):
    # Null and NaN are the same in Pandas :)
    return df.isnull().any()


def isnull_values_sum(df):
    return df.isnull().values.sum() > 0


def isnull_sum(df):
    return df.isnull().sum() > 0


def isnull_values_any(df):
    return df.isnull().values.any()


def rows_with_null(df):
    return df[df.isnull().any(axis=1)]


def get_tmin_tmax(start, duration, end_cutoff):
    return (start - end_cutoff, start + duration - end_cutoff)


def to_numpy_reshape(x):
    return DataFrame.to_numpy(x).reshape(-1, 1)


def get_cnt_filename(i_user: int, state: str):
    return "{i_user}_{state}.cnt".format(i_user=i_user, state=state)


def glimpse_df(df: DataFrame):

    print("\nShowing first 3 data points\n")
    display(df.head(n=3))

    print("\nShowing last 3 data points\n")
    display(df.tail(n=3))

    print("\nShowing 3 radnom data points\n")
    display(df.sample(n=3))

    display(df.describe())


def powerset(iterable):
    "[1,2,3] --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(0, len(s) + 1))


def get_dictionary_leaves(dictionary: dict):
    """
    {a: 3, b: {c: 3, d: 4}} --> [[a,3], [c,3], [d,4]]
    """

    def get_leaves(pair):
        key, value = pair
        if type(value) is dict:
            return get_dictionary_leaves(value)
        return [[key, value]]

    result = []
    for pair in dictionary.items():
        result.extend(get_leaves(pair))
    return result


def dict_to_byte_metadata(dictionary: dict):
    """
    {a:3, b:2, c:test} ---> "a 3, b 2, c test"
    """
    pairs = get_dictionary_leaves(dictionary)
    return ",".join(map(lambda key_value: " ".join([str(key_value[0]), str(key_value[1])]), pairs)).encode()


def dict_to_string(dictionary: dict):
    """
    {accuracy: 73, method: "net} ---> "accuracy=73___method=net"
    """
    pairs = get_dictionary_leaves(dictionary)
    return "__".join(map(lambda key_value: "=".join([str(key_value[0]), str(key_value[1])]), pairs))


class Tee(object):
    def __init__(self, *files):
        self.files = files

    def write(self, obj):
        for f in self.files:
            f.write(obj)
            f.flush()

    def flush(self):
        for f in self.files:
            f.flush()


def stdout_to_file(file: Path):
    f = open(Path(getcwd(), file), "w")
    sys.stdout = Tee(sys.stdout, f)

In [6]:
def fuzzy_entropy(x):
    return eh.FuzzEn(x, m=2, r=(np.std(x, ddof=0) * 0.2, 1))[0][-1]


def sample_entropy(x):
    return an.sample_entropy(x)


# don't normalize because you have to normalze across all users and not based on 1 user and 1 sample
def spectral_entropy(x, freq: float):
    return an.spectral_entropy(x, sf=freq, normalize=False)


def approximate_entropy(x):
    return an.app_entropy(x, order=2)


def pd_fuzzy_entropy(x: Series, standardize_input=False) -> float:
    # standardization doesnt affect result!
    x_np = x.to_numpy()
    if standardize_input:
        x_np = standard_scaler_1d(x_np)
    return fuzzy_entropy(x_np)


def pd_sample_entropy(x: Series, standardize_input=False) -> float:
    # standardization doesnt affect result!
    x_np = x.to_numpy()
    if standardize_input:
        x_np = standard_scaler_1d(x_np)
    return sample_entropy(x_np)


def pd_spectral_entropy(x: Series, freq: float, standardize_input=False) -> float:
    # standardization doesnt affect result!
    x_np = x.to_numpy()
    if standardize_input:
        x_np = standard_scaler_1d(x_np)
    return spectral_entropy(x_np, freq)


def pd_approximate_entropy(x: Series, standardize_input=False) -> float:
    x_np = x.to_numpy()
    if standardize_input:
        x_np = min_max_scaler_1d(x_np)
    return approximate_entropy(x_np)


In [ ]:
df_checkpoint = None
is_complete_train = True
if is_complete_train:
    print("Performing complete training")
num_users =  num_users
signal_duration = SIGNAL_DURATION_SECONDS_DEFAULT
epoch_elems = FREQ

train_metadata = {
    "is_complete_train": is_complete_train,
    "num_users": num_users,
    "sig_seconds": signal_duration,
    "epoch_elems": epoch_elems,
}

def signal_to_epochs(filename: str):
    """
    Load the signal.
    Exclude bad channels.
    Crops the filter the signal.
    Return epoches.

    Notes:
    eeg = read_raw_cnt(filename, eog=["HEOL", "HEOR", "VEOU", "VEOL"], preload=True, verbose=False

    when comparing with and without eog, it changed data dramatically? this is what allowed me to sync data with S
    """
    print(filename)
    eeg = read_raw_cnt(filename, preload=True, verbose=False)
    eeg.info["bads"].extend(channels_bad)
    eeg.pick_channels(channels_good)

    signal_total_duration = floor(len(eeg) / FREQ)
    start = signal_total_duration - signal_duration + signal_offset
    end = signal_total_duration + signal_offset
    eeg_filtered = eeg.crop(tmin=start, tmax=end).notch_filter(50).filter(l_freq=0.15, h_freq=40)

    return make_fixed_length_epochs(eeg_filtered, duration=EPOCH_SECONDS, preload=True, verbose=False)


def epochs_to_dataframe(epochs: Epochs):
    """
    Returns epochs converted to dataframe.
    Useless columns are excluded.
    """
    df: DataFrame = epochs.to_data_frame(scalings=dict(eeg=1))
    df = df.drop(["time", "condition", *channels_ignore], axis=1)
    return df


def normalize_df(df: DataFrame, columns_to_scale: list):
    """
    Normalizes dataframe by replacing values and scaling them.
    Standard scaler scales for each column independently.
    """
    df = df.replace([np.inf, -np.inf], np.nan)
    df[df <= 0] = 0
    df = df.fillna(0)
    df[columns_to_scale] = min_max_scaler(df[columns_to_scale])
    return df


if df_checkpoint:
    """
    If checkpoint only action to perform is normalizing since entropies are already caculated
    """
    df = normalize_df(read_pickle(Path(df_checkpoint)), entropy_channel_combinations)
    save_df_to_disk(df, train_metadata, PATH_DATA_DATAFRAME, "normalized")
    print("Only cleaning of existing df was performed.")
    sys.exit(1)

npy_matrix = np.zeros(shape=(len(states), num_users, len(entropy_names), signal_duration, len(channels_good)))
rows = []
for user_id in range(0, num_users):
    for state in states:
        print(user_id, state)
        file_signal = str(Path(PATH_DATASET_CNT, get_cnt_filename(user_id + 1, state)))
        epochs = signal_to_epochs(file_signal)
        df = epochs_to_dataframe(epochs)
        label = 1 if state == FATIGUE_STR else 0

        for epoch_id in range(0, signal_duration):
            """
            Filter dataframe rows that have the current epoch are selected.
            Caculate entropy array for all channels. Shape (30,)
            Create a simple dictionary and use to return append entropies in a proper.
            Order of entropies is defined by list entropy_names.

            Append to backup matrix
            Append to list that contains the label and properly ordered entropies
            e.g. [0, PE_FP1, PE_FP2, ... , PE_C3, AE_FP1, AE_FP2, ..., FE_C3]
            """

            df_dict = {}
            df_epoch = df.loc[df["epoch"] == epoch_id].head(epoch_elems)
            df_channels = df_epoch[channels_good]

            df_spectral_entropy = df_channels.apply(func=lambda x: pd_spectral_entropy(x, freq=FREQ, standardize_input=True), axis=0)
            df_approximate_entropy = df_channels.apply(func=lambda x: pd_approximate_entropy(x, standardize_input=True), axis=0)
            df_sample_entropy = df_channels.apply(func=lambda x: pd_sample_entropy(x, standardize_input=True), axis=0)
            df_fuzzy_entropy = df_channels.apply(func=lambda x: pd_fuzzy_entropy(x, standardize_input=True), axis=0)

            df_dict = {
                "PE": df_spectral_entropy,
                "AE": df_approximate_entropy,
                "SE": df_sample_entropy,
                "FE": df_fuzzy_entropy,
            }

            for i, e in enumerate(entropy_names):
                npy_matrix[label][user_id][i][epoch_id] = np.array(df_dict[entropy_names[i]])

            rows.append([label, user_id, epoch_id, *df_dict[entropy_names[0]], *df_dict[entropy_names[1]], *df_dict[entropy_names[2]], *df_dict[entropy_names[3]]])

        if is_complete_train:
            np.save(str(Path(PATH_DATA_DATAFRAME, ".raw_matrix")), npy_matrix)

"""Create dataframe from rows and columns"""
columns = ["label", "user_id", "epoch_id"] + entropy_channel_combinations
df = DataFrame(rows, columns=columns)
df["label"] = df["label"].astype(int)

"""Complete training - save instantly so no error with naming is possible"""
if is_complete_train:
    np.save(str(Path(PATH_DATA_DATAFRAME, ".raw_npy.npy")), npy_matrix)
    df.to_pickle(str(Path(PATH_DATA_DATAFRAME, ".raw_df.pkl")))

"""Save to files"""
save_npy_to_disk(npy_matrix, PATH_DATA_DATAFRAME, "npy_matrix", train_metadata)
save_df_to_disk(df, is_complete_train, PATH_DATA_DATAFRAME, "raw-with-userid", train_metadata)
df = normalize_df(df, entropy_channel_combinations)
glimpse_df(df)
print(save_df_to_disk(df, is_complete_train, PATH_DATA_DATAFRAME, "normalized-with-userid", train_metadata))
df = df.drop(["user_id", "epoch_id"])
print(save_df_to_disk(df, is_complete_train, PATH_DATA_DATAFRAME, "normalized", train_metadata))


In [ ]:
wide_params = sorted([*np.logspace(-6, 3, 10), 500])  # from 1e-6 to 1000, mutiplying by ten in between

svm_parameters = [{"gamma": sorted([1e-3, 1e-4, PAPER_G]), "C": sorted([PAPER_C, 100, 500, 1000, 1500])}]
svm_parameters_wide = [{"gamma": wide_params, "C": wide_params}]
mlp_parameters = {"alpha": [0.00001, 0.0001, 0.001, 0.05], "learning_rate": ["constant", "adaptive"]}
rfc_parameters = {"n_estimators": [PAPER_RFC_TREES], "max_features": [PAPER_RFC_INPUT_VARIABLES, "auto"]}
knn_parameters = {"weights": ["uniform", "distance"]}

model_rfc = GridSearchCV(RandomForestClassifier(), rfc_parameters)
model_mlp = GridSearchCV(MLPClassifier(activation="logistic", hidden_layer_sizes=PAPER_BF_HIDDEN, max_iter=500), mlp_parameters)
model_svc = GridSearchCV(SVC(kernel="rbf"), svm_parameters)
model_svc_wide = GridSearchCV(SVC(kernel="rbf"), svm_parameters_wide)
model_knn = GridSearchCV(KNeighborsClassifier(), knn_parameters)


In [25]:
df_file = Path(PATH_DATA_DATAFRAME,"complete-normalized-with_user_id-2021-12-08-16-10-19-.pkl")
timestamp = get_timestamp()
report_filename = Path(PATH_REPORT, "-".join(["loo-parameters", timestamp]) + ".txt")
print(report_filename)
stdout_to_file(report_filename)

df = read_pickle(df_file)
glimpse_df(df)
X = df.loc[:, ~df.columns.isin(["label"])]
y = df.loc[:, df.columns.isin(["label", "user_id"])]
training_columns = X.columns.isin(entropy_channel_combinations)

groups = X["user_id"].to_numpy()
acc_parameters = []

for C, gamma in tqdm(list(product(wide_params, wide_params))):
    """
    Itterating over the grid of parameters C and gamma to find the best combination
    """
    model = SVC(kernel="rbf", C=C, gamma=gamma)
    acc_total = 0

    for train_index, test_index in LeaveOneGroupOut().split(X, y, groups):
        """
        LeaveOneGroupOut where the "group" of rows is actaully a single participant.
        Effectivly, we are training on 11 participants and testing our accuracy on 1 participant
        """
        X_train, X_test = X.iloc[train_index, training_columns], X.iloc[test_index, training_columns]
        y_train, y_test = y.iloc[train_index]["label"], y.iloc[test_index]["label"]
        model.fit(X_train, y_train)
        y_test_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_test_pred)
        acc_total += acc
    acc_parameters.append([acc_total / num_users, C, gamma])

print("Acc\t\t\tC\tgamma")
accs = sorted(acc_parameters, key=lambda x: x[0], reverse=True)
for acc in accs:
    print(acc)


,label,PE_FP1,PE_FP2,PE_F7,PE_F3,PE_FZ,PE_F4,PE_F8,PE_FT7,PE_FC3,...,FE_TP8,FE_T5,FE_P3,FE_PZ,FE_P4,FE_T6,FE_O1,FE_OZ,FE_O2,user_id
0,0,0.209317,0.294891,0.133174,0.065747,-0.116481,0.063046,0.144465,0.042837,0.082721,...,-0.817313,-0.946274,-0.807386,-0.077408,-0.812991,0.137097,-0.831627,-0.086549,-0.958774,0
1,0,-0.268841,-0.213876,-0.058541,-0.352989,-0.290602,-0.354629,-0.337714,-0.366898,-0.339992,...,-0.991332,-0.997530,-0.895272,-0.140533,-0.898464,0.059291,-0.992012,0.048274,-0.922727,0
2,0,-0.264437,-0.213709,-0.321779,-0.352851,-0.421904,-0.354492,-0.342027,-0.366763,-0.378028,...,-0.988322,-0.996644,-0.830196,-0.037422,-0.850739,0.186383,-0.989237,-0.368149,-0.810525,0


/home/matej/2-fer/uuzop/eeg-driver-fatigue-detection/data/reports/loo-parameters-2021-12-09-02-44-39.txt


,label,PE_FP1,PE_FP2,PE_F7,PE_F3,PE_FZ,PE_F4,PE_F8,PE_FT7,PE_FC3,...,FE_TP8,FE_T5,FE_P3,FE_PZ,FE_P4,FE_T6,FE_O1,FE_OZ,FE_O2,user_id
7197,1,-0.223214,-0.213739,-0.186230,-0.352876,-0.117740,0.340319,-0.203473,-0.366787,-0.339246,...,-0.988319,-0.996643,-0.754175,-0.997432,-0.409622,-0.996912,-1.0,-0.413888,-0.997353,11
7198,1,-0.284287,-0.213903,-0.147141,-0.353011,-0.020959,-1.000000,-0.400705,-0.366919,-0.384937,...,-0.991331,-0.997530,-0.705023,-0.998095,-0.692108,-0.997728,-1.0,-0.386990,-0.998036,11
7199,1,-0.040386,0.145975,0.138400,-0.056817,-0.056981,0.363188,-0.055777,-0.077092,-0.016836,...,-0.800899,-0.941440,-0.578164,-0.956236,-0.525546,-0.946134,-1.0,-0.126120,-0.954907,11


,label,PE_FP1,PE_FP2,PE_F7,PE_F3,PE_FZ,PE_F4,PE_F8,PE_FT7,PE_FC3,...,FE_TP8,FE_T5,FE_P3,FE_PZ,FE_P4,FE_T6,FE_O1,FE_OZ,FE_O2,user_id
6520,1,0.059589,-1.000000,-0.111721,-1.0,-0.243107,-1.0,-0.430824,-1.0,-0.336354,...,-0.999717,-1.0,0.385077,-0.999938,0.305821,0.282748,-0.99974,-0.011322,-0.999936,10
2903,1,0.299270,-0.952114,0.233381,-1.0,0.212064,-1.0,0.313031,-1.0,-0.071722,...,-0.999717,-1.0,0.589288,-0.999938,0.608084,-0.885266,-0.99974,0.184431,-0.999936,4
1933,0,-0.450883,-1.000000,0.458336,-1.0,0.328628,-1.0,0.200127,-1.0,0.244167,...,-0.999717,-1.0,0.287107,-0.999938,0.270542,-0.997027,-0.99974,0.149953,-0.999936,3


,label,PE_FP1,PE_FP2,PE_F7,PE_F3,PE_FZ,PE_F4,PE_F8,PE_FT7,PE_FC3,...,FE_TP8,FE_T5,FE_P3,FE_PZ,FE_P4,FE_T6,FE_O1,FE_OZ,FE_O2,user_id
count,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,...,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000
mean,0.500000,0.089403,-0.903883,0.098153,-0.724004,-0.022082,-0.690965,-0.025598,-0.729406,-0.097114,...,-0.970908,-0.918479,0.332845,-0.779727,0.285013,-0.753079,-0.972363,0.046798,-0.976073,5.500000
std,0.500035,0.314508,0.276742,0.368863,0.453024,0.340669,0.484015,0.310877,0.472049,0.308131,...,0.125191,0.296989,0.362547,0.424517,0.371594,0.511292,0.131415,0.320503,0.152111,3.452292
min,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,0.000000,-0.088150,-1.000000,-0.167985,-1.000000,-0.271564,-1.000000,-0.263067,-1.000000,-0.309987,...,-0.999717,-1.000000,0.166007,-0.999938,0.099338,-1.000000,-0.999740,-0.132894,-0.999936,2.750000
50%,0.500000,0.147971,-1.000000,0.160845,-1.000000,0.020586,-1.000000,0.033686,-1.000000,-0.055816,...,-0.999717,-1.000000,0.438100,-0.999938,0.372477,-1.000000,-0.999740,0.094102,-0.999936,5.500000
75%,1.000000,0.335433,-1.000000,0.399038,-0.354412,0.265769,-0.354550,0.239478,-0.383018,0.158110,...,-0.999717,-0.999828,0.595382,-0.900736,0.574105,-0.957479,-0.999740,0.271222,-0.999936,8.250000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.000000


  1%|          | 1/121 [02:43<5:27:07, 163.56s/it]

In [11]:
if 'report_filename' not in locals():
    report_filename = Path(PATH_REPORT, "loo-parameters-2021-12-08-23-25-17.txt")
for line in open(report_filename, "r").readlines():
	print(line)

Results

Acc			C	gamma

[0.5, 1e-06, 10.0]

[0.5, 1e-05, 10.0]

[0.5, 0.0001, 10.0]

[0.5, 0.0001, 100.0]

[0.5, 0.001, 10.0]

[0.5, 0.001, 100.0]

[0.5, 0.01, 10.0]

[0.5, 0.01, 100.0]

[0.5, 0.1, 10.0]

[0.5, 0.1, 100.0]

[0.5, 1.0, 10.0]

[0.5, 1.0, 100.0]

[0.5, 10.0, 100.0]

[0.5, 100.0, 100.0]

[0.5, 500, 100.0]

[0.5, 1000.0, 100.0]

[0.4997222222222222, 10.0, 10.0]

[0.4997222222222222, 100.0, 10.0]

[0.4997222222222222, 500, 10.0]

[0.4997222222222222, 1000.0, 10.0]

[0.4959722222222222, 1e-06, 1000.0]

[0.4959722222222222, 1e-05, 1000.0]

[0.49583333333333335, 0.0001, 1000.0]

[0.49569444444444444, 0.001, 1000.0]

[0.4955555555555555, 0.01, 1000.0]

[0.4955555555555555, 0.1, 1000.0]

[0.4955555555555555, 1.0, 1000.0]

[0.4955555555555555, 10.0, 1000.0]

[0.4955555555555555, 100.0, 1000.0]

[0.4955555555555555, 500, 1000.0]

[0.4955555555555555, 1000.0, 1000.0]

[0.4877777777777778, 1.0, 500]

[0.4877777777777778, 10.0, 500]

[0.4877777777777778, 100.0, 500]

[0.48777777777777

In [12]:
df_file = Path(PATH_DATA_DATAFRAME,"complete-normalized-with_user_id-2021-12-08-16-10-19-.pkl")
timestamp = get_timestamp()
report_ent_filename = Path(PATH_REPORT, "-".join(["best-entropies", timestamp]) + ".txt")
stdout_to_file(report_ent_filename)


df: DataFrame = read_pickle(df_file)
X = df.loc[:, ~df.columns.isin(["label"])]
y = df.loc[:, "label"]

X_train_org, X_test_org, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

entropy_excluded_powerset = list(powerset(entropy_names))[:-1]  # Exclude last element where all entropies are mentioned
models = [model_svc]
scorings = ["accuracy"]
results = []

for i, pair in enumerate(tqdm(list(product(scorings, models, entropy_excluded_powerset)))):
    scoring, model, entropies_exclude = pair
    (X_train, X_test) = (X_train_org.copy(), X_test_org.copy())

    for entropy in entropies_exclude:
        X_train = X_train.loc[:, ~X_train.columns.str.startswith(entropy)]
        X_test = X_test.loc[:, ~X_test.columns.str.startswith(entropy)]

    model.scoring = scoring
    model.fit(X_train, y_train)

    y_true_train, y_pred_train = y_train, model.predict(X_train)
    y_true_test, y_pred_test = y_test, model.predict(X_test)

    classification_report_string = classification_report(y_true_test, y_pred_test, digits=6, output_dict=True)

    results.append([i, list(set(entropy_names) - set(entropies_exclude)), model.best_score_, get_dictionary_leaves(classification_report_string)])

for result in sorted(results, key=lambda x: x[2], reverse=True):
    print(result, "\n")


  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
report_ent_filename = Path(PATH_DATA_MODEL, "report_best_entropies-2021-12-08-14-48-27.txt")
for line in open(report_ent_filename, "r").readlines():
	print(line)

In [6]:
df_file = Path(PATH_DATA_DATAFRAME,"complete-normalized-with_user_id-2021-12-08-16-10-19-.pkl")
timestamp = get_timestamp()
report_ent_filename = Path(PATH_REPORT, "-".join(["best-entropies", timestamp]) + ".txt")
stdout_to_file(report_ent_filename)


df_file = Path(PATH_DATA_DATAFRAME,"complete-normalized-with_user_id-2021-12-08-16-10-19-.pkl")
df = read_pickle(df_file)
glimpse_df(df)

df = df.loc[:, df.columns.isin(["label", *entropy_channel_combinations])]
X = df.loc[:, ~df.columns.isin(["label"])]
y = df.loc[:, "label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

scorings = ["accuracy"]  # scorings = ["accuracy", "f1"]
models = [model_rfc, model_mlp, model_knn, model_svc]
for pair in product(scorings, models):
    scoring, model = pair

    model_name = type(model.estimator).__name__
    model.scoring = scoring
    model.fit(X_train, y_train)
    save_model(model=model, model_name=model_name, score=model.best_score_, directory=PATH_DATA_MODEL, metadata=model.best_params_)

    print("=== Best model {} with accuracy {} and parameters {}\n\n".format(model_name, model.best_score_, model.best_params_))
    print("Grid scores on test set:\n")
    means = model.cv_results_["mean_test_score"]
    stds = model.cv_results_["std_test_score"]
    for mean, std, params in sorted(zip(means, stds, model.cv_results_["params"]), key=lambda x: x[0]):
        print("%0.6f (+/-%0.6f) for %r" % (mean, std * 2, params))

    y_true_train, y_pred_train = y_train, model.predict(X_train)
    y_true_test, y_pred_test = y_test, model.predict(X_test)

    print("\nReport on train set:")
    classification_report_string = classification_report(y_true_train, y_pred_train, digits=6)
    print(classification_report_string)

    print("Report on test set:")
    classification_report_string = classification_report(y_true_test, y_pred_test, digits=6)
    print(classification_report_string)

glimpse_df(df)



Showing first 3 data points



,label,PE_FP1,PE_FP2,PE_F7,PE_F3,PE_FZ,PE_F4,PE_F8,PE_FT7,PE_FC3,...,FE_TP8,FE_T5,FE_P3,FE_PZ,FE_P4,FE_T6,FE_O1,FE_OZ,FE_O2,user_id
0,0,0.209317,0.294891,0.133174,0.065747,-0.116481,0.063046,0.144465,0.042837,0.082721,...,-0.817313,-0.946274,-0.807386,-0.077408,-0.812991,0.137097,-0.831627,-0.086549,-0.958774,0
1,0,-0.268841,-0.213876,-0.058541,-0.352989,-0.290602,-0.354629,-0.337714,-0.366898,-0.339992,...,-0.991332,-0.997530,-0.895272,-0.140533,-0.898464,0.059291,-0.992012,0.048274,-0.922727,0
2,0,-0.264437,-0.213709,-0.321779,-0.352851,-0.421904,-0.354492,-0.342027,-0.366763,-0.378028,...,-0.988322,-0.996644,-0.830196,-0.037422,-0.850739,0.186383,-0.989237,-0.368149,-0.810525,0



Showing last 3 data points



,label,PE_FP1,PE_FP2,PE_F7,PE_F3,PE_FZ,PE_F4,PE_F8,PE_FT7,PE_FC3,...,FE_TP8,FE_T5,FE_P3,FE_PZ,FE_P4,FE_T6,FE_O1,FE_OZ,FE_O2,user_id
7197,1,-0.223214,-0.213739,-0.186230,-0.352876,-0.117740,0.340319,-0.203473,-0.366787,-0.339246,...,-0.988319,-0.996643,-0.754175,-0.997432,-0.409622,-0.996912,-1.0,-0.413888,-0.997353,11
7198,1,-0.284287,-0.213903,-0.147141,-0.353011,-0.020959,-1.000000,-0.400705,-0.366919,-0.384937,...,-0.991331,-0.997530,-0.705023,-0.998095,-0.692108,-0.997728,-1.0,-0.386990,-0.998036,11
7199,1,-0.040386,0.145975,0.138400,-0.056817,-0.056981,0.363188,-0.055777,-0.077092,-0.016836,...,-0.800899,-0.941440,-0.578164,-0.956236,-0.525546,-0.946134,-1.0,-0.126120,-0.954907,11



Showing 3 radnom data points



,label,PE_FP1,PE_FP2,PE_F7,PE_F3,PE_FZ,PE_F4,PE_F8,PE_FT7,PE_FC3,...,FE_TP8,FE_T5,FE_P3,FE_PZ,FE_P4,FE_T6,FE_O1,FE_OZ,FE_O2,user_id
158,0,-0.184344,-1.000000,-0.268709,-1.00000,-0.029334,-1.0,-0.155428,-0.318624,0.166951,...,-0.999717,-0.997708,0.693088,0.140540,0.506213,0.405736,-0.99974,-0.113937,-0.988199,0
2747,1,0.290713,-0.229381,0.542583,-1.00000,0.401464,-1.0,0.360373,-1.000000,0.157470,...,-0.979737,-0.999888,0.584358,-0.999719,0.593716,-0.468809,-0.99974,0.499922,-0.999936,4
4879,0,-0.068762,-1.000000,-0.442296,0.51397,-0.553710,-1.0,-0.602066,-0.383615,-0.776535,...,-0.999717,-0.340068,0.680518,0.028959,0.662486,-1.000000,-0.99974,0.403044,-0.991886,8


,label,PE_FP1,PE_FP2,PE_F7,PE_F3,PE_FZ,PE_F4,PE_F8,PE_FT7,PE_FC3,...,FE_TP8,FE_T5,FE_P3,FE_PZ,FE_P4,FE_T6,FE_O1,FE_OZ,FE_O2,user_id
count,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,...,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000
mean,0.500000,0.089403,-0.903883,0.098153,-0.724004,-0.022082,-0.690965,-0.025598,-0.729406,-0.097114,...,-0.970908,-0.918479,0.332845,-0.779727,0.285013,-0.753079,-0.972363,0.046798,-0.976073,5.500000
std,0.500035,0.314508,0.276742,0.368863,0.453024,0.340669,0.484015,0.310877,0.472049,0.308131,...,0.125191,0.296989,0.362547,0.424517,0.371594,0.511292,0.131415,0.320503,0.152111,3.452292
min,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,0.000000,-0.088150,-1.000000,-0.167985,-1.000000,-0.271564,-1.000000,-0.263067,-1.000000,-0.309987,...,-0.999717,-1.000000,0.166007,-0.999938,0.099338,-1.000000,-0.999740,-0.132894,-0.999936,2.750000
50%,0.500000,0.147971,-1.000000,0.160845,-1.000000,0.020586,-1.000000,0.033686,-1.000000,-0.055816,...,-0.999717,-1.000000,0.438100,-0.999938,0.372477,-1.000000,-0.999740,0.094102,-0.999936,5.500000
75%,1.000000,0.335433,-1.000000,0.399038,-0.354412,0.265769,-0.354550,0.239478,-0.383018,0.158110,...,-0.999717,-0.999828,0.595382,-0.900736,0.574105,-0.957479,-0.999740,0.271222,-0.999936,8.250000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.000000


In [ ]:
report_ent_filename = Path(PATH_DATA_MODEL, "train-models-2021-12-09-19-17-19.txt")
for line in open(report_ent_filename, "r").readlines():
	print(line)